## Module import 

In [1]:
from tensorboardX import SummaryWriter
summary = SummaryWriter()

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
torch.manual_seed(777) # randomseed 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda': torch.cuda.manual_seed_all(777)

## data 불러오기

In [3]:
# 데이터가 저장된 경로 설정
data_path = '../data/'

# 데이터 불러오기
train = pd.read_csv(data_path + 'train_df.csv', encoding='cp949')
test = pd.read_csv(data_path + 'test_df.csv', encoding='cp949')
submission = pd.read_csv(data_path + 'sample_submission.csv')

# **카테고리 열 라벨인코딩**

In [4]:
category_columns = ['SEND_SPG_INNB', 'DL_GD_LCLS_NM', 'DL_GD_MCLS_NM']
for col in category_columns:
    le = LabelEncoder()
    le.fit(pd.concat([train[col], test[col]], axis=0))
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

In [5]:
send = pd.concat([train['SEND_SPG_INNB'], test['SEND_SPG_INNB']], axis=0).max()+1
lc = pd.concat([train['DL_GD_LCLS_NM'], test['DL_GD_LCLS_NM']], axis=0).max()+1
mc = pd.concat([train['DL_GD_MCLS_NM'], test['DL_GD_MCLS_NM']], axis=0).max()+1
send, lc, mc

(3983, 6, 20)

# **Train/Valid 데이터 생성**

In [6]:
x_train = train.drop(['index', 'REC_SPG_INNB' ,'INVC_CONT'], axis=1)
y_train = train['INVC_CONT'].values
x_test = test.drop(['index', 'REC_SPG_INNB'], axis=1)

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=777)

In [8]:
x_train = torch.LongTensor(np.array(x_train)).to(device)
y_train = torch.FloatTensor(np.array(y_train)).to(device).reshape(-1,1)
x_val = torch.LongTensor(np.array(x_val)).to(device)
y_val = torch.FloatTensor(np.array(y_val)).to(device).reshape(-1,1)
x_test = torch.LongTensor(np.array(x_test)).to(device)

# **Train/Valid Loader**

In [9]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)

val_dataset = TensorDataset(x_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True, drop_last=True)

# **DNN Model & Training**

In [ ]:
class DNN(torch.nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        K = 150
        self.send = torch.nn.Embedding(send, K)
        self.lc = torch.nn.Embedding(lc, K)
        self.mc = torch.nn.Embedding(mc, K)

        
        self.relu = torch.nn.ReLU()

        self.Linear1 = torch.nn.Linear(450, 1024)
        self.Linear2 = torch.nn.Linear(1024, 2048)
        self.Linear3 = torch.nn.Linear(2048, 4096)
        self.Linear4 = torch.nn.Linear(4096, 2048)
        self.Linear5 = torch.nn.Linear(2048, 1024)
        self.Linear6 = torch.nn.Linear(1024, 512)
        self.Linear7 = torch.nn.Linear(512, 1)
        
        
        self.bn1 = torch.nn.BatchNorm1d(1024)
        self.bn2 = torch.nn.BatchNorm1d(2048)
        self.bn3 = torch.nn.BatchNorm1d(4096)
        self.bn4 = torch.nn.BatchNorm1d(2048)
        self.bn5 = torch.nn.BatchNorm1d(1024)
        self.bn6 = torch.nn.BatchNorm1d(512)
        
        self.model = torch.nn.Sequential(self.Linear1, self.bn1 ,self.relu,# self.drop,
                                         self.Linear2, self.bn2 ,self.relu,# self.drop,
                                         self.Linear3, self.bn3 ,self.relu,# self.drop,
                                         self.Linear4, self.bn4 ,self.relu,# self.drop,
                                         self.Linear5, self.bn5 ,self.relu,# self.drop,
                                         self.Linear6, self.bn6 ,self.relu,# self.drop,
                                         self.Linear7)
        self.weight_init()
    def weight_init(self):
    # model에 있는 모듈을 하나씩 불러와 Linear 모듈이면 xavier로 weight 초기화
        for module in self.model:
            if isinstance(module, torch.nn.Linear):
                torch.nn.init.xavier_uniform_(module.weight)
                module.bias.data.fill_(0.01)

    def forward(self, x):
        # Tensor 입력을 받아서 forwarding 진행
        SEND = torch.flatten(self.send(x[:,0:1]),1)
        LC = torch.flatten(self.send(x[:,1:2]),1)
        MC = torch.flatten(self.send(x[:,2:3]),1)
        # LC = torch.flatten(self.lc(x[:,1:2]),1)
        # MC = torch.flatten(self.mc(x[:,2:3]),1)
        R = torch.cat([SEND, LC, MC], axis=1)

        return self.model(R)

model = DNN().to(device)

epochs = 10
# epochs = 12
batch = 3200
lr = 5e-5

optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=1e-6)
loss_fn = torch.nn.MSELoss().to(device)

for epoch in range(epochs+1):
    avg_loss = 0
    model.train()
    for X, Y in tqdm(train_loader):
        hypothesis = model(X)
        loss = loss_fn(hypothesis, Y)
        loss = loss**0.5
        avg_loss += loss/len(train_loader)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        avg_val_loss = 0
    for X_val, Y_val in tqdm(val_loader):
        val_h = model(x_val)
        loss_val = loss_fn(val_h, y_val)
        loss_val = loss_val**0.5
        avg_val_loss += loss_val / len(val_loader)
    if epoch % 1 == 0:
        print("Epoch : {:4d} Train_Loss : {:.7f} Val_Loss : {:.7f}".format(epoch, avg_loss, avg_val_loss))
        summary.add_scalar('Train_loss', avg_loss.item(), epoch)
        summary.add_scalar('Validation_loss', avg_val_loss, epoch)

  0%|          | 0/900 [00:00<?, ?it/s]

# **Train/Valid Loss 시각화**

In [11]:
%load_ext tensorboard
%tensorboard --logdir runs --port=6006

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

# **Test**

In [12]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
predict

tensor([[5.6416],
        [4.9382],
        [5.4238],
        ...,
        [4.2793],
        [4.1109],
        [3.7777]], device='cuda:0')

# **Base Line에 사용된 LGBMRegressor**

In [ ]:
from lightgbm import LGBMRegressor
model_ = LGBMRegressor()
model_.fit(x_train.to('cpu'),y_train.to('cpu').reshape(-1),verbose=True)
model_.predict(x_test.to('cpu'))

array([5.08556398, 7.03933874, 5.27872916, ..., 4.58522293, 4.81370918,
       4.51272358])

# **Submit 파일 생성**

In [ ]:
submit = pd.read_csv("/content/drive/MyDrive/DACON/물류유통량예측/sample_submission.csv")
submit

,index,INVC_CONT
0,32000,0
1,32001,0
2,32002,0
3,32003,0
4,32004,0
...,...,...
4635,36635,0
4636,36636,0
4637,36637,0
4638,36638,0


In [ ]:
submit['INVC_CONT'] = np.array(predict.cpu())
submit

,index,INVC_CONT
0,32000,5.681759
1,32001,5.156948
2,32002,5.641582
3,32003,5.641582
4,32004,4.532369
...,...,...
4635,36635,4.183407
4636,36636,4.507492
4637,36637,4.239439
4638,36638,4.527795


In [ ]:
submit.to_csv("/content/drive/MyDrive/DACON/물류유통량예측/submit.csv",index=False)

# **Embedding 오류**
Embedding 하는 과정에서 모두 self.send로 임베딩을 하는 실수를 하였는데
각 피쳐에 맞게 self.lc, self.mc로 임베딩하고 epoch를 12로 설정하면
public : 5.465195434
private : 8.276577598
의 score가 나옴